<a href="https://colab.research.google.com/github/KienLe21/Transformers_Testing/blob/main/Demo_qa_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### DEMO QUESTION ANSWERING
Demo Question Answering model với pre-trained model là distilbert và dataset là SQuAD

* Tải các thư viện cần thiết

In [1]:
! pip install transformers datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.2 MB/s eta 0:00:00


* Đăng nhập vào huggingface_hub để lưu lại model sau khi train

In [2]:
from huggingface_hub import notebook_login

notebook_login()

* Lấy dataset SQuAD

In [3]:
from datasets import load_dataset

squad = load_dataset("squad", split="train[:15000]")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

* Chia data ra thành tập hai tập data train và test

In [4]:
squad = squad.train_test_split(test_size=0.2)

* Kiểm tra data

In [5]:
squad["train"][0]

{'id': '56de37c0cffd8e1900b4b687',
 'title': 'Southern_Europe',
 'context': 'The Roman Empire came to dominate the entire Mediterranean basin in a vast empire based on Roman law and Roman legions. It promoted trade, tolerance, and Greek culture. By 300 AD the Roman Empire was divided into the Western Roman Empire based in Rome, and the Eastern Roman Empire based in Constantinople. The attacks of the Germanic peoples of northern Europe led to the Fall of the Western Roman Empire in AD 476, a date which traditionally marks the end of the classical period and the start of the Middle Ages.',
 'question': 'What year is considered the transition between the classical period and the Middle Ages?',
 'answers': {'text': ['AD 476'], 'answer_start': [409]}}

* Tokenized data


In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [7]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [8]:
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [9]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

* Training model với pre-trained model `distilbert`

In [10]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
training_args = TrainingArguments(
    output_dir="demo_qa_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-11-76e7a5ccc295>:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kien-le2152004 (kien-le2152004-ho-chi-minh-city-university-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss
1,2.799700,1.484060
2,1.239700,1.325213
3,0.976200,1.342809


TrainOutput(global_step=2250, training_loss=1.5617983262803818, metrics={'train_runtime': 1366.4647, 'train_samples_per_second': 26.345, 'train_steps_per_second': 1.647, 'total_flos': 3527633700864000.0, 'train_loss': 1.5617983262803818, 'epoch': 3.0})

In [12]:
trainer.push_to_hub()

events.out.tfevents.1740018280.1a01a1505a0c.351.0:   0%|          | 0.00/6.94k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/KienLe21/demo_qa_model/commit/ea36eebac17fd4903cd6e5b88d63577a7a6690a2', commit_message='End of training', commit_description='', oid='ea36eebac17fd4903cd6e5b88d63577a7a6690a2', pr_url=None, repo_url=RepoUrl('https://huggingface.co/KienLe21/demo_qa_model', endpoint='https://huggingface.co', repo_type='model', repo_id='KienLe21/demo_qa_model'), pr_revision=None, pr_num=None)

* Đánh giá


In [13]:
question = "What is the capital of France?"
context = "France is a country in Western Europe known for its rich history, art, and culture. Its capital, Paris, is famous for landmarks like the Eiffel Tower and the Louvre Museum."


In [15]:
question = "Who discovered gravity?"
context = "Gravity is a fundamental force that keeps objects on Earth. It was first studied in depth by Sir Isaac Newton in the 17th century after he observed an apple falling from a tree."


In [17]:
question = "How many planets are there in the Solar System?"
context = "The Solar System consists of the Sun and the celestial bodies that orbit it, including eight recognized planets: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune."


In [19]:
question = "What year did the Titanic sink?"
context = "The RMS Titanic, one of the largest and most luxurious ships of its time, tragically sank in the North Atlantic Ocean on April 15, 1912, after hitting an iceberg."


In [21]:
question = "What is the chemical symbol for water?"
context = "Water is essential for life and is composed of two hydrogen atoms and one oxygen atom. Its chemical formula is H₂O, which signifies its molecular structure."


In [ ]:
question = "What were the primary factors leading to the decline of the Roman Empire?"
context = "The decline of the Roman Empire was attributed to a combination of internal weaknesses and external pressures. Internally, political instability, economic struggles, and military corruption weakened the empire’s core. The division between the Eastern and Western Roman Empires, initiated by Diocletian, also contributed to administrative inefficiencies. Externally, barbarian invasions, most notably the sacking of Rome by the Visigoths in 410 AD and the final collapse after the deposition of Romulus Augustulus in 476 AD, marked the definitive end of the Western Roman Empire."


In [ ]:
question = "How does quantum entanglement challenge classical physics?"
context = "Quantum entanglement is a phenomenon where two or more particles become correlated such that the state of one instantly influences the state of the other, regardless of distance. This contradicts classical physics' principle of locality, which states that objects are only influenced by their immediate surroundings. The famous Einstein-Podolsky-Rosen (EPR) paradox and subsequent Bell's theorem experiments provided strong evidence that quantum mechanics allows for non-local interactions, challenging traditional notions of causality and separability in physics."


In [ ]:
question = "What are the key economic factors contributing to hyperinflation in a country?"
context = "Hyperinflation occurs when a country experiences extremely high and typically accelerating inflation. The primary causes include excessive money supply growth, often due to government overprinting of currency to finance deficits. Demand-pull inflation, where aggregate demand outstrips supply, also contributes. Additionally, loss of confidence in the currency leads to rapid depreciation, as seen in historical cases like Zimbabwe and Weimar Germany, where prices doubled within days, rendering money virtually worthless."


In [ ]:
question = "What role does DNA methylation play in gene expression regulation?"
context = "DNA methylation is a crucial epigenetic modification that regulates gene expression by adding methyl groups to cytosine residues, primarily in CpG islands. This process typically results in transcriptional repression, as it alters chromatin structure and prevents transcription factors from binding to DNA. Aberrant DNA methylation patterns are associated with various diseases, including cancer, where hypermethylation can silence tumor suppressor genes while hypomethylation may activate oncogenes."


In [ ]:
question = "How does the greenhouse effect contribute to global climate change?"
context = "The greenhouse effect is a natural process where certain gases in Earth's atmosphere, such as carbon dioxide, methane, and water vapor, trap heat from the Sun. This trapped heat helps maintain Earth's temperature at a level suitable for life. However, human activities, including the burning of fossil fuels and deforestation, have significantly increased the concentration of greenhouse gases, enhancing the greenhouse effect. This leads to rising global temperatures, more extreme weather patterns, melting ice caps, and disruptions to ecosystems worldwide."


In [14]:
from transformers import AutoModelForQuestionAnswering, pipeline

model_name = "KienLe21/demo_qa_model"

model = AutoModelForQuestionAnswering.from_pretrained(model_name)

config.json:   0%|          | 0.00/561 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cuda:0


{'score': 0.9478123784065247, 'start': 97, 'end': 102, 'answer': 'Paris'}

In [24]:
demo_qa = pipeline("question-answering", model=model_name)
demo_qa(question=question, context=context)

Device set to use cuda:0


{'score': 0.9174241423606873, 'start': 111, 'end': 114, 'answer': 'H₂O'}

### Thử nghiệm train với model `deepset/roberta-base-squad2`

In [23]:
roberta_base_model = AutoModelForQuestionAnswering.from_pretrained("deepset/roberta-base-squad2")

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

In [25]:
training_args = TrainingArguments(
    output_dir="roberta_qa_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-25-1c526c7e2619>:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,0.867300,1.428176
2,0.523400,1.470231
3,0.396800,1.622922


TrainOutput(global_step=2250, training_loss=0.5845602213541666, metrics={'train_runtime': 1487.6558, 'train_samples_per_second': 24.199, 'train_steps_per_second': 1.512, 'total_flos': 3527633700864000.0, 'train_loss': 0.5845602213541666, 'epoch': 3.0})

In [26]:
trainer.push_to_hub()

events.out.tfevents.1740020397.1a01a1505a0c.351.1:   0%|          | 0.00/6.98k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/KienLe21/roberta_qa_model/commit/33e5b962e2d10e6a0ddfd7c5b900981292758360', commit_message='End of training', commit_description='', oid='33e5b962e2d10e6a0ddfd7c5b900981292758360', pr_url=None, repo_url=RepoUrl('https://huggingface.co/KienLe21/roberta_qa_model', endpoint='https://huggingface.co', repo_type='model', repo_id='KienLe21/roberta_qa_model'), pr_revision=None, pr_num=None)